In [ ]:
Ensembles = ['../input/sub-files/Submission files/sub (1).csv','../input/sub-files/Submission files/sub (2).csv','../input/sub-files/Submission files/sub (3).csv',
            '../input/sub-files/Submission files/sub (4).csv','../input/sub-files/Submission files/sub (5).csv','../input/sub-files/Submission files/sub (6).csv',
            '../input/sub-files/Submission files/sub (7).csv','../input/sub-files/Submission files/sub (8).csv','../input/sub-files/Submission files/sub (9).csv',
            '../input/sub-files/Submission files/sub (10).csv','../input/sub-files/Submission files/sub (11).csv','../input/sub-files/Submission files/sub (12).csv',
            '../input/sub-files/Submission files/sub (13).csv','../input/sub-files/Submission files/sub (14).csv','../input/sub-files/Submission files/sub (15).csv',
            '../input/sub-files/Submission files/sub (16).csv','../input/sub-files/Submission files/sub (17).csv','../input/sub-files/Submission files/sub (18).csv',
            '../input/sub-files/Submission files/sub (19).csv','../input/sub-files/Submission files/sub (20).csv','../input/sub-files/Submission files/sub (21).csv',
            '../input/sub-files/Submission files/sub (22).csv',]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from sklearn.model_selection import KFold,GroupKFold
from tensorflow.keras import layers

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

targets = train['pressure'].to_numpy().reshape(-1, 80, 1)

In [ ]:
unique_pressures = np.unique(targets)

sorted_pressures = np.sort(unique_pressures)

PRESSURE_STEP = (unique_pressures[1] - unique_pressures[0]).item()

PRESSURE_MIN = sorted_pressures[0].item()
PRESSURE_MAX = sorted_pressures[-1].item()

In [ ]:
test_preds = []

for predicted in Ensembles:
    submission_file = pd.read_csv(predicted)
    test_preds.append(submission_file['pressure'].values.reshape(-1,1))

In [ ]:
predictions = np.concatenate(test_preds, axis=-1)

In [ ]:
predictions = np.array(list(map(lambda i : np.dot(i, i.T)/i.sum(), predictions)))

In [ ]:
#median_pre = np.median(predictions, axis = -1)
rounding_pre = np.round((predictions - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
clipped_pre = np.clip(rounding_pre, PRESSURE_MIN, PRESSURE_MAX)

In [ ]:
submission_file = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
submission_file['pressure'] = clipped_pre.reshape(-1, )
submission_file.to_csv('submission.csv', index=False)